In [1]:
import typing as T  # pylint: disable=unused-import

import numpy as np
import numpy.typing as npt

from pydrake.solvers import (  # pylint: disable=import-error, no-name-in-module, unused-import
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
    MosekSolver,
    MosekSolverDetails,
    SolverOptions,
    CommonSolverOption,
)
from pydrake.geometry.optimization import (  # pylint: disable=import-error, no-name-in-module
    HPolyhedron,
    Point,
    ConvexSet,
)

from pydrake.symbolic import ( # pylint: disable=import-error, no-name-in-module, unused-import
    Polynomial,
    Variable,
    Variables,
    Expression,
)  
from program_options import ProgramOptions, FREE_POLY, PSD_POLY

from util import (
    timeit,
    diditwork,
    INFO,
    YAY,
    WARN,
    ERROR,
)  # pylint: disable=import-error, no-name-in-module, unused-import

import plotly.graph_objects as go  # pylint: disable=import-error
from plotly.express.colors import sample_colorscale  # pylint: disable=import-error
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from polynomial_gcs_dual import PolynomialDualGCS, Vertex, Edge
from kink_example import random_uniform_graph_generator, build_m_step_horizon_from_layers, plot_m_step_horizon_from_layers, plot_a_layered_graph, plot_policy_rollout

In [2]:
def add_rollout_plots(fig, gcs, layers, y_true, lookahead, vertex, layer_index, color, dx=0.1, subtract_nsteps:bool = False,use_0_potentials:bool=False, plot_lb=True):
    nsteps_subtract = 0
    if subtract_nsteps:
        nsteps_subtract = len(layers)-1
    if plot_lb:        
        x,y = plot_m_step_horizon_from_layers(gcs, layers, lookahead, vertex, layer_index,dx=dx)
        fig.add_trace(go.Scatter(x=x, y=y-nsteps_subtract, mode='lines', name=str(lookahead)+"-step-lookahead lb", line=dict(color=color) ))
    x_r, y_r = plot_policy_rollout(gcs, layers, lookahead, vertex, layer_index,dx=dx, use_0_potentials=use_0_potentials)
    fig.add_trace(go.Scatter(x=x_r, y=y_r-nsteps_subtract, mode='lines', name=str(lookahead)+"-step-lookahead ub", line=dict(color=color) ))
    
    if plot_lb:
        lower_bound_gap = (1 - (y-nsteps_subtract)/(y_true-nsteps_subtract))*100
        upper_bound_gap = ((y_r-nsteps_subtract)/(y_true-nsteps_subtract) - 1)*100
        INFO(str(lookahead)+"-step-lookahead statistics")
        INFO("max lower bound gap ", np.round(np.max(lower_bound_gap),2))
        INFO("mean lower bound gap ", np.round(np.mean(lower_bound_gap),2))
        INFO("max upper bound gap ", np.round(np.max(upper_bound_gap),2))
        INFO("mean upper bound gap ", np.round(np.mean(upper_bound_gap),2))
        INFO("---------------------")


In [16]:
options = ProgramOptions()
number_of_layers = 15
options.mosek_tolerance_gap = 1e-9
options.mosek_primal_feas_gap = 1e-9
options.mosek_dual_feas_gap = 1e-9
options.max_constraint_degree=2
# gcs, layers = random_uniform_graph_generator(num_layers=15, options=options,x_min=0, x_max=15, random_seed=2, min_blank=1, max_blank=5, min_goal_blank=2, max_goal_blank=5)
gcs, layers = random_uniform_graph_generator(num_layers=number_of_layers, options=options,x_min=0, x_max=15, random_seed=2, min_blank=1, max_blank=5, min_goal_blank=2, max_goal_blank=5)
fig = plot_a_layered_graph(layers)
fig.show()
fig.update_layout(height=1200, width=1200)

dx = 0.2

v_0 = layers[0][0]
fig, y_true = gcs.make_plots(v_0.name, dx=dx, subtract_nsteps=0)


add_rollout_plots(fig, gcs, layers, y_true, 1, v_0, 0, "red", dx=dx, use_0_potentials=True,plot_lb=True)
add_rollout_plots(fig, gcs, layers, y_true, 2, v_0, 0, "orange", dx=dx, use_0_potentials=True,plot_lb=True)

add_rollout_plots(fig, gcs, layers, y_true, 1, v_0, 0, "purple", dx=dx, use_0_potentials=False,plot_lb=True)
add_rollout_plots(fig, gcs, layers, y_true, 2, v_0, 0, "magenta", dx=dx, use_0_potentials=False,plot_lb=True)
fig.update_layout(height=1000, width=1000)
fig.update_yaxes(range=[4, 24])   # Set y-axis limits
fig.update_traces(line={'width': 5})
fig.show()

Solve took 0.124s
solve successful!
-112.74779792882013
SolutionResult.kSolutionFound
Solver is Mosek
time 0.0921180248260498
rescode 0
solution_status 1


polynomial policy statistics
max lower bound gap  43.37
mean lower bound gap  16.04
---------------------
1-step-lookahead statistics
max lower bound gap  25.38
mean lower bound gap  5.62
max upper bound gap  125.63
mean upper bound gap  60.7
---------------------
2-step-lookahead statistics
max lower bound gap  16.06
mean lower bound gap  2.72
max upper bound gap  114.94
mean upper bound gap  55.87
---------------------
1-step-lookahead statistics
max lower bound gap  25.38
mean lower bound gap  5.62
max upper bound gap  29.41
mean upper bound gap  2.89
---------------------
2-step-lookahead statistics
max lower bound gap  16.06
mean lower bound gap  2.72
max upper bound gap  8.51
mean upper bound gap  0.42
---------------------


In [15]:
fig = plot_a_layered_graph(layers)
fig.show()
fig.update_yaxes(range=[4, 24])   # Set y-axis limits
fig.update_traces(line={'width': 5})

In [2]:
options = ProgramOptions()
options.use_convex_relaxation = False

graph = PolynomialDualGCS(options)
# test out on something simpel

quad_cost = lambda x, y: np.sum([(x[i]-y[i])**2 for i in range(len(x))])

a1 = graph.AddTargetVertex("a1", Point([0]), lambda x: 0 )
b1 = graph.AddVertex("b1", HPolyhedron.MakeBox([-3],[-1]))
b2 = graph.AddVertex("b2", HPolyhedron.MakeBox([1],[3]))
c1 = graph.AddVertex("c1", HPolyhedron.MakeBox([-3],[3]))

graph.AddEdge(c1, b1, quad_cost)
graph.AddEdge(c1, b2, quad_cost)
graph.AddEdge(b1, a1, quad_cost)
graph.AddEdge(b2, a1, quad_cost)

for v in graph.gcs.Edges():
    print(v.name())

graph.solve_for_true_shortest_path("c1", [2])
graph.solve_for_true_shortest_path("c1", [3])


e0
c1 b1
c1 b2
b1 a1
b2 a1


TypeError: PolynomialDualGCS.solve_for_true_shortest_path() missing 1 required positional argument: 'point'

: 

In [7]:
for v in graph.gcs.Edges():
    print(v.name())

e5
c1 b1
c1 b2
b1 a1
b2 a1
e5
e6
e7
e8


In [3]:
prog = MathematicalProgram()
x = prog.NewIndeterminates(3, "x_")
xt = prog.NewIndeterminates(3, "xt_")
vars = Variables(np.hstack((x, xt)))

<Variables "{x_(0), x_(1), x_(2), xt_(0), xt_(1), xt_(2)}">